In [71]:
from string import ascii_lowercase

class AlphabetTransformer:
    def __init__(self):
        self._letter_to_number = {}
        self._number_to_letter = {}
        cnt = 0
        for c in ascii_lowercase:
            self._letter_to_number[c] = cnt
            self._number_to_letter[cnt] = c
            cnt += 1
        self._size = cnt

        
    def encrypt(self, c, key):
        shift = self._letter_to_number[key]
        res = (self._letter_to_number[c] + shift) % self._size
        return self._number_to_letter[res]
    
    def decrypt(self, c, key):
        shift = self._letter_to_number[key]
        res = (self._letter_to_number[c] + self._size - shift) % self._size
        return self._number_to_letter[res]
    
    def get_diff(self, a, b):
        return self._letter_to_number[b] - self._letter_to_number[a]

In [6]:
transformer = AlphabetTransformer()

for c in ascii_lowercase:
    for d in ascii_lowercase:
        print(transformer.encrypt(c, d).upper(), end='')
    print()

ABCDEFGHIJKLMNOPQRSTUVWXYZ
BCDEFGHIJKLMNOPQRSTUVWXYZA
CDEFGHIJKLMNOPQRSTUVWXYZAB
DEFGHIJKLMNOPQRSTUVWXYZABC
EFGHIJKLMNOPQRSTUVWXYZABCD
FGHIJKLMNOPQRSTUVWXYZABCDE
GHIJKLMNOPQRSTUVWXYZABCDEF
HIJKLMNOPQRSTUVWXYZABCDEFG
IJKLMNOPQRSTUVWXYZABCDEFGH
JKLMNOPQRSTUVWXYZABCDEFGHI
KLMNOPQRSTUVWXYZABCDEFGHIJ
LMNOPQRSTUVWXYZABCDEFGHIJK
MNOPQRSTUVWXYZABCDEFGHIJKL
NOPQRSTUVWXYZABCDEFGHIJKLM
OPQRSTUVWXYZABCDEFGHIJKLMN
PQRSTUVWXYZABCDEFGHIJKLMNO
QRSTUVWXYZABCDEFGHIJKLMNOP
RSTUVWXYZABCDEFGHIJKLMNOPQ
STUVWXYZABCDEFGHIJKLMNOPQR
TUVWXYZABCDEFGHIJKLMNOPQRS
UVWXYZABCDEFGHIJKLMNOPQRST
VWXYZABCDEFGHIJKLMNOPQRSTU
WXYZABCDEFGHIJKLMNOPQRSTUV
XYZABCDEFGHIJKLMNOPQRSTUVW
YZABCDEFGHIJKLMNOPQRSTUVWX
ZABCDEFGHIJKLMNOPQRSTUVWXY


In [9]:
class Encryptor:
    def __init__(self, key):
        self._key = key
        self._transformer = AlphabetTransformer()
        
    def _get_shift(self, word):
        shift = self._key
        while len(shift) < len(word):
            shift += self._key
        return shift
    
    def encrypt(self, word):
        shift = self._get_shift(word)
        res = [self._transformer.encrypt(c, k) for c, k in zip(word, shift)]
        return ''.join(res)
        
    def decrypt(self, word):
        shift = self._get_shift(word)
        res = [self._transformer.decrypt(c, key=k) for c, k in zip(word, shift)]
        return ''.join(res)

In [10]:
encryptor = Encryptor('lemon')
word = 'fgfctcdtrydcutdqedxhfdiyg'
cipher = encryptor.encrypt(word)
assert word == encryptor.decrypt(cipher)

In [260]:
from collections import Counter, defaultdict
from math import gcd
from functools import reduce
import re
import itertools

LETTER_FREQUENCY = {'a': 8.2, 'b': 1.5,  'c': 2.8, 'd': 4.3, 'e': 13, 'f': 2.2, 'g': 2, 
                    'h': 6.1, 'i': 7, 'j' : 0.15, 'k': 0.77, 'l': 4, 'm': 2.4,  'n': 6.7, 'o': 7.5,
                    'p': 1.9, 'q': 0.095, 'r': 6,  's': 6.3, 't': 9.1, 'u': 2.8,  'v': 0.98, 'w': 2.4, 'x': 0.15, 
                    'y': 2, 'z': 0.074}
    
class Decryptor: 
    def __init__(self, text):
        self._text = text
        self._substr_frequency = defaultdict(Counter)
        self._possible_lengths = []
        self._transformer = AlphabetTransformer()
        
        
    def _compute_substr_frequencies(self):
        for k in range(7, 1, -1):
            for i in range(len(self._text) - k):
                self._substr_frequency[k].update([self._text[i:i + k]])
            self._substr_frequency[k].update([self._text[-k:]])
        
    def _get_key_sizes(self, substr):
        indices = [m.start() for m in re.finditer(substr, self._text)]
        diffs = [indices[i] - indices[i - 1] for i in range(1, len(indices))]
        #print('Diffs', diffs)
        key_length = reduce(gcd, diffs)
        return [i for i in range(key_length, 0, -1) if key_length % i == 0]
    
    def _get_key_from_length(self, length):
        #print(length)
        key = []
        for i in range(length):
            counter = Counter([self._text[j] for j in range(len(self._text)) if j % length == i])
            total = sum(counter.values())
            min_diff = 1e9
            for shift in ascii_lowercase:
                dist = {self._transformer.decrypt(c, shift): k / total * 100 for c, k in counter.most_common(26)}
                abs_diff = 0
                for l in ascii_lowercase:
                    if l not in dist:
                        dist[l] = 0
                    abs_diff += abs(dist[l] - LETTER_FREQUENCY[l])
                if abs_diff < min_diff:
                    min_diff = abs_diff
                    res = shift
            #print(f'POSSIBLE RESULT FOR POSITION {i}: {res}')
            key.append(res)
        return key

    def compute(self):
        self._compute_substr_frequencies()
        for k in range(7, 1, -1):
            #print(self._substr_frequency[k].most_common(10))
            for s, count in self._substr_frequency[k].most_common(10):
                if count < 5:
                    continue
                for length in self._get_key_sizes(s):
                    if length == 1:
                        continue
                    #print(length)
                    key = self._get_key_from_length(length)
                    yield ''.join(key)
        yield self._get_key_from_length(1)

In [248]:
decr = Decryptor('ABCDEFGHK')

decr._compute_substr_frequencies()
decr._substr_frequency

defaultdict(collections.Counter,
            {7: Counter({'ABCDEFG': 1, 'BCDEFGH': 1, 'CDEFGHK': 1}),
             6: Counter({'ABCDEF': 1, 'BCDEFG': 1, 'CDEFGH': 1, 'DEFGHK': 1}),
             5: Counter({'ABCDE': 1,
                      'BCDEF': 1,
                      'CDEFG': 1,
                      'DEFGH': 1,
                      'EFGHK': 1}),
             4: Counter({'ABCD': 1,
                      'BCDE': 1,
                      'CDEF': 1,
                      'DEFG': 1,
                      'EFGH': 1,
                      'FGHK': 1}),
             3: Counter({'ABC': 1,
                      'BCD': 1,
                      'CDE': 1,
                      'DEF': 1,
                      'EFG': 1,
                      'FGH': 1,
                      'GHK': 1}),
             2: Counter({'AB': 1,
                      'BC': 1,
                      'CD': 1,
                      'DE': 1,
                      'EF': 1,
                      'FG': 1,
                      '

In [256]:
import string

def prepare_text(input_str):
    input_str = input_str.lower()
    chars = set(input_str)
    chars_to_remove = ''
    for c in chars:
        if c not in string.ascii_lowercase:
            chars_to_remove += c
    return input_str.translate(str.maketrans(
        '', '', chars_to_remove))

In [257]:
example_text = '''The North Wind and the Sun had a quarrel about which of them was the stronger. While they were disputing with much heat and bluster, a Traveler passed along the road wrapped in a cloak."Let us agree," said the Sun, "that he is the stronger who can strip that Traveler of his cloak.""Very well," growled the North Wind, and at once sent a cold, howling blast against the Traveler.

With the first gust of wind the ends of the cloak whipped about the Traveler's body. But he immediately wrapped it closely around him, and the harder the Wind blew, the tighter he held it to him. The North Wind tore angrily at the cloak, but all his efforts were in vain.

Then the Sun began to shine. At first his beams were gentle, and in the pleasant warmth after the bitter cold of the North Wind, the Traveler unfastened his cloak and let it hang loosely from his shoulders. The Sun's rays grew warmer and warmer. The man took off his cap and mopped his brow. At last he became so heated that he pulled off his cloak, and, to escape the blazing sunshine, threw himself down in the welcome shade of a tree by the roadside.

Gentleness and kind persuasion win where force and bluster fail. There was once a Countryman who possessed the most wonderful Goose you can imagine, for every day when he visited the nest, the Goose had laid a beautiful, glittering, golden egg.

The Countryman took the eggs to market and soon began to get rich. But it was not long before he grew impatient with the Goose because she gave him only a single golden egg a day. He was not getting rich fast enough.

Then one day, after he had finished counting his money, the idea came to him that he could get all the golden eggs at once by killing the Goose and cutting it open. But when the deed was done, not a single golden egg did he find, and his precious Goose was dead.

Those who have plenty want more and so lose all they have. The Tortoise, you know, carries his house on his back. No matter how hard he tries, he cannot leave home. They say that Jupiter punished him so, because he was such a lazy stay-at-home that he would not go to Jupiter's wedding, even when especially invited.

After many years, Tortoise began to wish he had gone to that wedding. When he saw how gaily the birds flew about and how the Hare and the Chipmunk and all the other animals ran nimbly by, always eager to see everything there was to be seen, the Tortoise felt very sad and discontented. He wanted to see the world too, and there he was with a house on his back and little short legs that could hardly drag him along.

One day he met a pair of Ducks and told them all his trouble.

"We can help you to see the world," said the Ducks. "Take hold of this stick with your teeth and we will carry you far up in the air where you can see the whole countryside. But keep quiet or you will be sorry."

The Tortoise was very glad indeed. He seized the stick firmly with his teeth, the two Ducks took hold of it one at each end, and away they sailed up toward the clouds.

Just then a Crow flew by. He was very much astonished at the strange sight and cried:

"This must surely be the King of Tortoises!"

"Why certainly??" began the Tortoise.

But as he opened his mouth to say these foolish words he lost his hold on the stick, and down he fell to the ground, where he was dashed to pieces on a rock.

Foolish curiosity and vanity often lead to misfortune'''

In [258]:
test_algorithm(example_text, 'kalina')

Found kalina on iteration 0
Found kalina on iteration 3
Found kalina on iteration 6
Found kalina on iteration 9
Found kalina on iteration 12
Found kalina on iteration 15
Found kalina on iteration 18
Found kalina on iteration 21
Found kalina on iteration 24


In [261]:
def test_algorithm(text_to_encrypt, key):
    text_to_encrypt = prepare_text(text_to_encrypt)
    #print(text_to_encrypt)
    cipher = Encryptor(key).encrypt(text_to_encrypt)
    decr = Decryptor(cipher)
    assert text_to_encrypt == Encryptor(key).decrypt(cipher)
    for i, k in enumerate(decr.compute()):
        #print(k)
        encr = Encryptor(k)
        if encr.decrypt(cipher) == text_to_encrypt:
            print(f'Found {k} on iteration {i}')

In [262]:
test_algorithm(example_text, 'lemon')

Found lemon on iteration 0
Found lemon on iteration 1
Found lemon on iteration 2
Found lemon on iteration 3
Found lemon on iteration 4
Found lemon on iteration 5
Found lemon on iteration 6
Found lemon on iteration 7
Found lemon on iteration 8
Found lemon on iteration 9
Found lemon on iteration 10
Found lemon on iteration 11
Found lemon on iteration 12


In [263]:
test_algorithm(example_text, 'like')

Found like on iteration 0
Found like on iteration 2
Found like on iteration 4
Found like on iteration 6
Found like on iteration 8
Found like on iteration 11
Found like on iteration 13
Found like on iteration 15
Found like on iteration 17
Found like on iteration 19
Found like on iteration 21
Found like on iteration 23
Found like on iteration 25
Found like on iteration 30


In [207]:
another_example_text = '''Donald John Trump (born June 14, 1946) is an American media personality and real estate developer who served as the 45th president of the United States from 2017 to 2021.

Born and raised in Queens, New York City, Trump attended Fordham University for two years and received a bachelor's degree in economics from the Wharton School of the University of Pennsylvania. He became the president of his father Fred Trump's real estate business in 1971, which he renamed The Trump Organization; he expanded the company's operations to building and renovating skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name. Trump and his businesses have been involved in more than 4,000 state and federal legal actions, including six bankruptcies. He owned the Miss Universe brand of beauty pageants from 1996 to 2015, and produced and hosted the reality television series The Apprentice from 2004 to 2015.

Trump's political positions have been described as populist, protectionist, isolationist, and nationalist. He entered the 2016 presidential race as a Republican and was elected in an upset victory over Democratic nominee Hillary Clinton while losing the popular vote.[a] He became the first U.S. president without prior military or government service. His election and policies have sparked numerous protests. Trump made many false and misleading statements during his campaigns and presidency, to a degree unprecedented in American politics. Many of his comments and actions have been characterized as racially charged or racist.

During his presidency, Trump ordered a travel ban on citizens from several Muslim-majority countries, citing security concerns; after legal challenges, the Supreme Court upheld the policy's third revision. He enacted a tax-cut package for individuals and businesses, rescinding the individual health insurance mandate penalty of the Affordable Care Act (ACA). He appointed Neil Gorsuch, Brett Kavanaugh, and Amy Coney Barrett to the Supreme Court. He reacted slowly to the COVID-19 pandemic, downplayed the threat, ignored or contradicted many recommendations from health officials, promoted false information about unproven treatments and the availability of testing and fumbled vaccine distribution. In foreign policy, Trump pursued an America First agenda: he renegotiated the North American Free Trade Agreement (NAFTA) as the United States–Mexico–Canada Agreement (USMCA), and withdrew the U.S. from the Trans-Pacific Partnership trade negotiations, the Paris Agreement on climate change, and the Iran nuclear deal. He imposed import tariffs which triggered a trade war with China, moved the U.S. embassy in Israel to Jerusalem, and withdrew U.S. troops from northern Syria. He met three times with North Korean leader Kim Jong-un, but negotiations on denuclearization eventually broke down.

A special counsel investigation led by Robert Mueller found that Trump and his campaign benefited from Russian interference in the 2016 presidential election, but did not find sufficient evidence to press charges of criminal conspiracy or coordination with Russia.[b] Mueller also investigated Trump for obstruction of justice, and his report neither indicted nor exonerated Trump on that offense. Trump later pardoned five people who were convicted as a result of the Russia investigation. After Trump solicited Ukraine to investigate his political rival Joe Biden, the House of Representatives impeached him in December 2019 for abuse of power and obstruction of Congress. The Senate acquitted him of both charges in February 2020.

Trump lost the 2020 presidential election to Biden but refused to concede defeat. He attempted to overturn the results by making false claims of electoral fraud, pressuring government officials, mounting dozens of unsuccessful legal challenges, and obstructing the presidential transition. Hours before the ceremonial counting of the electoral votes on January 6, 2021, Trump rallied his supporters and exhorted them to march to the Capitol, which they then stormed. Congress evacuated, and five people died[c] in the melee. Seven days later, the House of Representatives impeached him again, for "incitement of insurrection", making him the only federal officeholder in the history of the United States to be impeached twice.

Donald John Trump was born at Jamaica Hospital in New York City's Queens borough on June 14, 1946.[1][2] His parents were Frederick Christ Trump, a Bronx-born real estate developer whose parents were German immigrants from Kallstadt, and Mary Anne MacLeod Trump, a Scottish housewife from Tong. He grew up in the Jamaica Estates neighborhood of Queens and attended the Kew-Forest School from kindergarten through seventh grade.[3][4] At age 13, he was enrolled in the New York Military Academy, a private boarding school,[5] and in 1964, he enrolled at Fordham University. Two years later he transferred to the Wharton School of the University of Pennsylvania, graduating in May 1968 with a B.S. in economics.[6][7] The New York Times reported in 1973 and 1976 that he had graduated first in his class at Wharton, but he had never made the school's honor roll.[8] In 2015, Trump's lawyer Michael Cohen threatened Fordham University and the New York Military Academy with legal action if they released Trump's academic records.[9]

Military deferment
While in college, Trump obtained four student draft deferments.[10] In 1966, he was deemed fit for military service based upon a medical examination, and in July 1968 a local draft board classified him as eligible to serve.[11] In October 1968, he was medically deferred and classified 1-Y (unqualified for duty except in the case of a national emergency).[12] In 1972, he was reclassified 4-F due to bone spurs, which permanently disqualified him from service.[13][14]

Family
Main article: Family of Donald Trump
Further information: Trump family
Parents and siblings
Main articles: Fred Trump and Mary Anne MacLeod Trump
Fred Trump started working in real estate with his mother Elizabeth when he was 15, after his father Friedrich had died in the 1918 flu pandemic.[15] By 1926, their company, "E. Trump & Son," was active in the New York boroughs of Queens and Brooklyn.[16] It would grow to build and sell tens of thousands of houses, barracks, and apartments.[17][18] Fred claimed to be Swedish amid the anti-German sentiment sparked by World War II;[19] Trump also claimed Swedish heritage until 1990.[20] Trump's mother Mary Anne MacLeod was born in Scotland.[21] Fred and Mary were married in 1936 and raised their family in Queens.[20] Trump grew up with three elder siblings – Maryanne, Fred Jr., and Elizabeth – and younger brother Robert.[22]

Wives and children

Trump is sworn in as president by Chief Justice John Roberts on January 20, 2017. From left: Trump, wife Melania, and his children Donald Jr., Barron, Ivanka, Eric, and Tiffany.
In 1977, Trump married Czech model Ivana Zelníčková.[23] They have three children, Donald Jr. (born 1977), Ivanka (born 1981), and Eric (born 1984), and ten grandchildren.[24] Ivana became a naturalized United States citizen in 1988.[25] The couple divorced in 1992, following Trump's affair with actress Marla Maples.[26] Maples and Trump married in 1993[27] and had one daughter, Tiffany (born 1993).[28] They were divorced in 1999,[29] and Tiffany was raised by Marla in California.[30] In 2005, Trump married Slovenian model Melania Knauss.[31] They have one son, Barron (born 2006).[32] Melania gained U.S. citizenship in 2006.[33]

Religion
Trump went to Sunday school and was confirmed in 1959 at the First Presbyterian Church in Jamaica, Queens.[34][35] In the 1970s, his parents joined the Marble Collegiate Church in Manhattan, which belongs to the Reformed Church.[34][36] The pastor at Marble, Norman Vincent Peale,[34] ministered to Trump's family until Peale's death in 1993.[36] Trump has described Peale as a mentor.[37] In 2015, after Trump said he attends Marble, the church stated he "is not an active member" of the church.[35] In November 2019, Trump appointed his personal pastor, televangelist Paula White, to the White House Office of Public Liaison.[38] In October 2020, Trump said that he identified as a non-denominational Christian.[39]

Wealth
Main article: Wealth of Donald Trump
See also: Tax returns of Donald Trump
In 1982, Trump was listed on the initial Forbes list of wealthy individuals as having a share of his family's estimated $200 million net worth. His financial losses in the 1980s caused him to be dropped from the list between 1990 and 1995.[40] In its 2020 billionaires ranking, Forbes estimated Trump's net worth at $2.1 billion[d] (1,001st in the world, 275th in the U.S.),[43] making him one of the richest politicians in American history and the first billionaire American president.[43] Forbes estimated that his net worth declined 31% and his ranking fell 138 spots between 2015 and 2018.[44] When he filed mandatory financial disclosure forms with the Federal Election Commission (FEC) in July 2015, Trump claimed a net worth of about $10 billion;[45] however, FEC figures cannot corroborate this estimate because they only show each of his largest buildings as being worth over $50 million, yielding total assets worth more than $1.4 billion and debt over $265 million.[46]


Trump and wife Ivana in the receiving line of a state dinner for King Fahd of Saudi Arabia in 1985,[47] with U.S. president Ronald Reagan and First Lady Nancy Reagan
Journalist Jonathan Greenberg reported in 2018 that Trump, using the pseudonym "John Barron" and claiming to be a Trump Organization official, called him in 1984 to falsely assert that he owned "in excess of ninety percent" of the Trump family's business, to secure a higher ranking on the Forbes 400 list of wealthy Americans. Greenberg also wrote that Forbes had vastly overestimated Trump's wealth and wrongly included him on the Forbes 400 rankings of 1982, 1983, and 1984.[48]

Trump has often said he began his career with "a small loan of one million dollars" from his father, and that he had to pay it back with interest.[49] In October 2018, The New York Times reported that Trump "was a millionaire by age 8," borrowed at least $60 million from his father, largely failed to repay those loans, and had received $413 million (adjusted for inflation) from his father's business empire over his lifetime.[50][51] According to the report, Trump and his family committed tax fraud, which a lawyer for Trump denied. The tax department of New York said it is investigating.[52][53] Trump's investments underperformed the stock market and the New York property market.[54][55] Forbes estimated in October 2018 that the value of Trump's personal brand licensing business had declined by 88% since 2015, to $3 million.[56]

Trump's tax returns from 1985 to 1994 show net losses totaling $1.17 billion over the ten-year period, in contrast to his claims about his financial health and business abilities. The New York Times reported that "year after year, Mr. Trump appears to have lost more money than nearly any other individual American taxpayer," and Trump's "core business losses in 1990 and 1991 – more than $250 million each year – were more than double those of the nearest taxpayers in the I.R.S. information for those years." In 1995 his reported losses were $915.7 million.[57][58]

According to a September 2020 analysis by The New York Times of twenty years of data from Trump's tax returns, Trump had accumulated hundreds of millions in losses, and deferred declaring $287 million in forgiven debt as taxable income.[59] According to the analysis, Trump's main sources of income were his share of revenue from The Apprentice and income from businesses in which he was a minority partner, while his majority-owned businesses were largely running at losses.[59] A significant portion of Trump's income was in tax credits due to his losses, which enables him to avoid paying income tax, or paying as little as $750, for several years.[59] Over the past decade, Trump has been balancing his businesses' losses by selling and taking out loans against assets, including a $100 million mortgage on Trump Tower (due in 2022) and the liquidation of over $200 million in stocks and bonds.[59] Trump has personally guaranteed $421 million in debt, most of which is due to be repaid by 2024. The tax records also showed Trump had unsuccessfully pursued business deals in China, including by developing a partnership with a major government-controlled company.[60]

Trump has a total of over $1 billion in debts, borrowed to finance his assets, reported Forbes in October 2020. Around $640 million or more was owed to various banks and trust organizations. Around $450 million was owed to unknown creditors. However, Trump's assets still outvalue his debts, reported Forbes.[61]

Business career
Main article: Business career of Donald Trump
Further information: Business projects of Donald Trump in Russia
Real estate

Trump Tower in Midtown Manhattan
While a student at Wharton and after graduating in 1968, Trump worked at his father Fred's real estate company, Trump Management, which owned middle-class rental housing in New York City's outer boroughs.[62][63][64] In 1971, he became president of the company and began using The Trump Organization as an umbrella brand.[65] It was registered as a corporation in 1981.[66]

Manhattan developments
Trump attracted public attention in 1978 with the launch of his family's first Manhattan venture, the renovation of the derelict Commodore Hotel, adjacent to Grand Central Terminal. The financing was facilitated by a $400 million city property tax abatement arranged by Fred Trump,[67] who also joined Hyatt in guaranteeing $70 million in bank construction financing.[68][69] The hotel reopened in 1980 as the Grand Hyatt Hotel,[70] and that same year, Trump obtained rights to develop Trump Tower, a mixed-use skyscraper in Midtown Manhattan.[71] The building houses the headquarters of the Trump Organization and was Trump's primary residence until 2019.[72][73]

In 1988, Trump acquired the Plaza Hotel in Manhattan with a loan of $425 million from a consortium of banks. Two years later, the hotel filed for bankruptcy protection, and a reorganization plan was approved in 1992.[74] In 1995, Trump lost the hotel to Citibank and investors from Singapore and Saudi Arabia, who assumed $300 million of the debt.[75][76]

In 1996, Trump acquired the vacant 71-story skyscraper at 40 Wall Street. After an extensive renovation, the high-rise was renamed the Trump Building.[77] In the early 1990s, Trump won the right to develop a 70-acre (28 ha) tract in the Lincoln Square neighborhood near the Hudson River. Struggling with debt from other ventures in 1994, Trump sold most of his interest in the project to Asian investors who were able to finance completion of the project, Riverside South.[78]

Palm Beach estate
Main article: Mar-a-Lago

Mar-a-Lago in 2009
In 1985, Trump acquired the Mar-a-Lago estate in Palm Beach, Florida.[79] Trump used a wing of the estate as a home, while converting the remainder into a private club with an initiation fee and annual dues.[80] In 2019, Trump declared Mar-a-Lago his primary residence,[73] although neighbors have indicated that Mar-a-Lago cannot be legally used as a residence.[81]

Atlantic City casinos
The entrance of the Trump Taj Mahal, a casino in Atlantic City. It has motifs evocative of the Taj Mahal in India.
Entrance of the Trump Taj Mahal in Atlantic City
In 1984, Trump opened Harrah's at Trump Plaza, a hotel and casino in Atlantic City, New Jersey. The project received financing from the Holiday Corporation, which also managed the operation. Gambling had been legalized there in 1977 to revitalize the once-popular seaside destination.[82] The property's poor financial results worsened tensions between Holiday and Trump, who paid Holiday $70 million in May 1986 to take sole control of the property.[83] Earlier, Trump had also acquired a partially completed building in Atlantic City from the Hilton Corporation for $320 million. Upon its completion in 1985, that hotel and casino were called Trump Castle. Trump's then-wife Ivana managed it until 1988.[84][85]

Trump acquired a third casino in Atlantic City, the Trump Taj Mahal, in 1988 in a highly leveraged transaction.[86] It was financed with $675 million in junk bonds and completed at a cost of $1.1 billion, opening in April 1990.[87][88][89] The project went bankrupt the following year,[88] and the reorganization left Trump with only half his initial ownership stake and required him to pledge personal guarantees of future performance.[90] Facing "enormous debt," he gave up control of his money-losing airline, Trump Shuttle, and sold his megayacht, the Trump Princess, which had been indefinitely docked in Atlantic City while leased to his casinos for use by wealthy gamblers.[91][92]

In 1995, Trump founded Trump Hotels & Casino Resorts (THCR), which assumed ownership of Trump Plaza, Trump Castle, and the Trump Casino in Gary, Indiana.[93] THCR purchased the Taj Mahal in 1996 and underwent successive bankruptcies in 2004, 2009, and 2014, leaving Trump with only ten percent ownership.[94] He remained chairman of THCR until 2009.[95]

Golf courses
Main article: Donald Trump and golf
The Trump Organization began acquiring and constructing golf courses in 1999.[96] It owned 16 golf courses and resorts worldwide and operated another two as of December 2016.[97]

From his inauguration until the end of 2019, Trump spent around one of every five days at one of his golf clubs.[98]

Branding and licensing
See also: List of things named after Donald Trump

Trump International Hotel and Tower in Chicago
The Trump name has been licensed for various consumer products and services, including foodstuffs, apparel, adult learning courses, and home furnishings.[99][100] According to an analysis by The Washington Post, there are more than fifty licensing or management deals involving Trump's name, which have generated at least $59 million in yearly revenue for his companies.[101] By 2018, only two consumer goods companies continued to license his name.[100]

Legal affairs and bankruptcies
Main articles: Legal affairs of Donald Trump and List of lawsuits involving Donald Trump
Fixer Roy Cohn served as Trump's lawyer and mentor for 13 years in the 1970s and 1980s.[102][103] According to Trump, Cohn sometimes waived fees due to their friendship.[63] In 1973, Cohn helped Trump countersue the United States government for $100 million over its charges that Trump's properties had racial discriminatory practices. Trump and Cohn lost that case when the countersuit was dismissed and the government's case went forward.[104] In 1975 an agreement was struck requiring Trump's properties to furnish the New York Urban League with a list of all apartment vacancies, every week, for two years, among other things.[105] Cohn introduced political consultant Roger Stone to Trump, who enlisted Stone's services to deal with the federal government.[106]

As of April 2018, Trump and his businesses had been involved in more than 4,000 state and federal legal actions, according to a running tally by USA Today.[107]

While Trump has not filed for personal bankruptcy, his over-leveraged hotel and casino businesses in Atlantic City and New York filed for Chapter 11 bankruptcy protection six times between 1991 and 2009.[108][109] They continued to operate while the banks restructured debt and reduced Trump's shares in the properties.[108][109]

During the 1980s, more than 70 banks had lent Trump $4 billion,[110] but in the aftermath of his corporate bankruptcies of the early 1990s, most major banks declined to lend to him, with only Deutsche Bank still willing to lend money.[111] The New York Times reported days after the 2021 storming of the United States Capitol that the bank had decided not to do business with Trump or his company in the future.[112]

In April 2019, the House Oversight Committee issued subpoenas seeking financial details from Trump's banks, Deutsche Bank and Capital One, and his accounting firm, Mazars USA. In response, Trump sued the banks, Mazars, and committee chairman Elijah Cummings to prevent the disclosures.[113][114] In May, DC District Court judge Amit Mehta ruled that Mazars must comply with the subpoena,[115] and judge Edgardo Ramos of the Southern District Court of New York ruled that the banks must also comply.[116][117] Trump's attorneys appealed the rulings,[118] arguing that Congress was attempting to usurp the "exercise of law-enforcement authority that the Constitution reserves to the executive branch."[119][120]

Side ventures

Trump attending a New York Mets baseball game at Citi Field
In September 1983, Trump purchased the New Jersey Generals, a team in the United States Football League. After the 1985 season, the league folded, largely due to Trump's strategy of moving games to a fall schedule (where they competed with the NFL for audience) and trying to force a merger with the NFL by bringing an antitrust suit against the organization.[121][122]

Trump's businesses have hosted several boxing matches at the Atlantic City Convention Hall adjacent to and promoted as taking place at the Trump Plaza in Atlantic City.[123][124] In 1989 and 1990, Trump lent his name to the Tour de Trump cycling stage race, which was an attempt to create an American equivalent of European races such as the Tour de France or the Giro d'Italia.[125]

In the late 1980s, Trump mimicked the actions of Wall Street's so-called corporate raiders. Trump began to purchase significant blocks of shares in various public companies, leading some observers to think he was engaged in the practice called greenmail, or feigning the intent to acquire the companies and then pressuring management to repurchase the buyer's stake at a premium. The New York Times found that Trump initially made millions of dollars in such stock transactions, but later "lost most, if not all, of those gains after investors stopped taking his takeover talk seriously."[57][126][127]

In 1988, Trump purchased the defunct Eastern Air Lines shuttle, with 21 planes and landing rights in New York City, Boston, and Washington, D.C. He financed the purchase with $380 million from 22 banks, rebranded the operation the Trump Shuttle, and operated it until 1992. Trump failed to earn a profit with the airline and sold it to USAir.[128]


Trump's star on the Hollywood Walk of Fame
In 1992, Trump, his siblings Maryanne, Elizabeth, and Robert, and cousin John W. Walter, each with a 20 percent share, formed All County Building Supply & Maintenance Corp. The company had no offices and is alleged to have been a shell company for paying the vendors providing services and supplies for Trump's rental units, and then billing those services and supplies to Trump Management with markups of 20–50 percent and more. The proceeds generated by the markups were shared by the owners.[51][129] The increased costs were used as justification to get state approval for increasing the rents of Trump's rent-stabilized units.[51]

From 1996 to 2015, Trump owned all or part of the Miss Universe pageants, including Miss USA and Miss Teen USA.[130][131] Due to disagreements with CBS about scheduling, he took both pageants to NBC in 2002.[132][133] In 2007, Trump received a star on the Hollywood Walk of Fame for his work as producer of Miss Universe.[134] After NBC and Univision dropped the pageants from their broadcasting lineups in June 2015,[135] Trump bought NBC's share of the Miss Universe Organization and sold the entire company to the William Morris talent agency.[130]'''

In [264]:
test_algorithm(another_example_text, 'lyrics')

Found lyrics on iteration 0
Found lyrics on iteration 3
Found lyricslyrics on iteration 6
Found lyrics on iteration 7
Found lyricslyrics on iteration 11
Found lyrics on iteration 12
Found lyrics on iteration 16
Found lyrics on iteration 19
Found lyrics on iteration 22
Found lyrics on iteration 25
Found lyrics on iteration 28
Found lyrics on iteration 31
Found lyrics on iteration 34
Found lyrics on iteration 37
Found lyrics on iteration 40
Found lyrics on iteration 43
Found lyrics on iteration 46
Found lyrics on iteration 49
Found lyrics on iteration 52
Found lyrics on iteration 55
Found lyrics on iteration 58
Found lyrics on iteration 61
Found lyrics on iteration 64
Found lyrics on iteration 67
Found lyrics on iteration 70
Found lyrics on iteration 73
Found lyrics on iteration 76
Found lyrics on iteration 79
Found lyrics on iteration 82
Found lyrics on iteration 85
Found lyrics on iteration 88
Found lyrics on iteration 91
Found lyrics on iteration 94
Found lyrics on iteration 97
Found 

In [265]:
test_algorithm(example_text, 'lyrics')

Found lyrics on iteration 0
Found lyrics on iteration 3
Found lyrics on iteration 6
Found lyrics on iteration 9
Found lyrics on iteration 12
Found lyrics on iteration 15


In [266]:
test_algorithm(example_text, 'bidon')

Found bidon on iteration 0
Found bidon on iteration 1
Found bidon on iteration 2
Found bidon on iteration 3
Found bidon on iteration 4
Found bidon on iteration 5
Found bidon on iteration 6
Found bidon on iteration 7
Found bidon on iteration 8
Found bidon on iteration 9
Found bidon on iteration 10


In [267]:
test_algorithm(example_text, 'asdfg')

Found asdfg on iteration 0
Found asdfg on iteration 1
Found asdfg on iteration 2
Found asdfg on iteration 3
Found asdfg on iteration 4
Found asdfg on iteration 5
Found asdfg on iteration 6
Found asdfg on iteration 7
Found asdfg on iteration 8


In [268]:
test_algorithm(another_example_text, 'qwert')

Found qwert on iteration 0
Found qwert on iteration 1
Found qwert on iteration 2
Found qwert on iteration 3
Found qwert on iteration 4
Found qwert on iteration 5
Found qwert on iteration 6
Found qwert on iteration 7
Found qwert on iteration 8
Found qwert on iteration 9
Found qwert on iteration 10
Found qwert on iteration 11
Found qwert on iteration 12
Found qwert on iteration 13
Found qwert on iteration 14
Found qwert on iteration 15
Found qwert on iteration 16
Found qwert on iteration 17
Found qwert on iteration 18
Found qwert on iteration 19
Found qwert on iteration 20
Found qwert on iteration 21
Found qwert on iteration 22
Found qwert on iteration 23
Found qwert on iteration 24
Found qwert on iteration 25
Found qwert on iteration 26
Found qwert on iteration 27
Found qwert on iteration 28
Found qwert on iteration 29
Found qwert on iteration 30
Found qwert on iteration 31
Found qwert on iteration 32
Found qwert on iteration 33
Found qwert on iteration 34
Found qwert on iteration 35
Fo

In [270]:
text_3 = '''For other uses, see True Detective (disambiguation).
True Detective
True Detective 2014 Intertitle.jpg
Title card for the first season
Genre	
Anthology
Crime drama
Detective
Mystery
Neo-noir
Created by	Nic Pizzolatto
Written by	
Nic Pizzolatto
Scott Lasser (s. 2)
David Milch (s. 3)
Graham Gordy (s. 3)
Starring	
Season 1
Matthew McConaughey
Woody Harrelson
Michelle Monaghan
Michael Potts
Tory Kittles
Season 2
Colin Farrell
Rachel McAdams
Taylor Kitsch
Kelly Reilly
Vince Vaughn
Season 3
Mahershala Ali
Carmen Ejogo
Stephen Dorff
Scoot McNairy
Ray Fisher
Opening theme	
"Far from Any Road" by The Handsome Family (s. 1)
"Nevermind" by Leonard Cohen (s. 2)
"Death Letter" by Cassandra Wilson (s. 3)
Composer	T Bone Burnett
Country of origin	United States
Original language	English
No. of seasons	3
No. of episodes	24 (list of episodes)
Production
Executive producers	
Nic Pizzolatto
Cary Joji Fukunaga
Scott Stephens
Matthew McConaughey
Woody Harrelson
Steve Golin
Richard Brown
Daniel Sackheim (s. 3)
Jeremy Saulnier (s. 3)
Bard Dorros (s. 3)
Producers	
Carol Cuddy (s. 1)
Aida Rodgers (s. 2)
Peter Feldman (s. 3)
Production locations	
Louisiana (s. 1)
California (s. 2)
Arkansas (s. 3)
Cinematography	
Adam Arkapaw (s. 1)
Nigel Bluck (s. 2–3)
Germain McMicking (s. 3)
Editors	
Alex Hall (s. 1–2)
Affonso Gonçalves (s. 1)
Meg Ritcher (s. 1)
Chris Fiegler (s. 2)
Byron Smith (s. 2)
Leo Trombetta (s. 3)
Running time	54–87 minutes
Production companies	
Anonymous Content
Parliament of Owls
Passenger
Neon Black
Lee Caplin / Picture Entertainment
HBO Entertainment
Distributor	Warner Bros. Television Distribution
Release
Original network	HBO
Original release	January 12, 2014 –
February 24, 2019
External links
Website
True Detective is an American anthology crime drama television series created and written by Nic Pizzolatto. The series, broadcast by the premium cable network HBO in the United States, premiered on January 12, 2014. Each season of the series is structured as a self-contained narrative, employing new cast ensembles, and following various sets of characters and settings.

The first season, starring Matthew McConaughey, Woody Harrelson, Michelle Monaghan, Michael Potts, and Tory Kittles, takes place in Louisiana and follows a pair of Louisiana State Police detectives, and their pursuit of a serial killer over a 17-year period. The second season aired in 2015, starring Colin Farrell, Rachel McAdams, Taylor Kitsch, Kelly Reilly, and Vince Vaughn,[1][2] is set in California, and focuses on three detectives from three cooperating police departments and a criminal-turned-businessman as they investigate a series of crimes they believe are linked to the murder of a corrupt politician. The third season aired in 2019, starring Mahershala Ali, Carmen Ejogo, Stephen Dorff, Scoot McNairy, and Ray Fisher, takes place in the Ozarks over three separate time periods as a pair of Arkansas State Police detectives investigate a macabre crime involving two missing children.

The first season received acclaim from critics and earned high ratings for HBO. It was nominated for and won numerous awards and other accolades, chiefly for its acting, cinematography, writing, and direction. Reception to the second season was more divided, although the show maintained high viewership for HBO. The third season received positive reviews, but saw a drop in viewership.


Contents
1	Production
1.1	Development
1.2	Filming
1.3	Opening sequence
2	Cast and crew
2.1	Season 1
2.2	Season 2
2.3	Season 3
2.4	Future
3	Episodes
4	Reception
4.1	Critical response
4.1.1	Season 1
4.1.2	Season 2
4.1.3	Season 3
4.2	Ratings
4.3	Accolades
5	Home media
6	References
7	External links
Production
Development
Before developing True Detective, Nic Pizzolatto worked as a literature professor for the University of Chicago, the University of North Carolina at Chapel Hill, and DePauw University.[3] He also delved into fiction writing, having developed a fascination for it as a graduate student at the University of Arkansas. His first published work was the short story collection Between Here and the Yellow Sea, released in 2006.[3] The author published his debut novel, titled Galveston, four years later and around the same time began preparing to branch out into the television industry (earlier attempts were never realized due to lack of capital).[4]

Intended to be Galveston's follow-up, Pizzolatto felt True Detective was more suited to film.[3][5] Pizzolatto shopped the novel to two TV executives, and, once he secured a deal in May 2010, drafted six screenplays, including the pilot episode ("The Long Bright Dark") script, which ran 90 pages.[3][4] He devoted another script for the series shortly after his departure from The Killing's writing staff in 2011, thanks to the support of Anonymous Content.[3] The final copy, amounting to 500 pages, was written without the aid of a writing staff.[6][7] By this time, Pizzolatto secured a development deal with HBO,[3] and by April 2012, the network commissioned True Detective on an order of eight episodes.[8] Set up as an anthology series, each season will feature a different cast of characters and self-contained narratives in various time periods and locations.[9]

Filming
The initial location for principal photography of True Detective's first season was Arkansas, but Pizzolatto later opted to film in Louisiana, which was cheaper due to its generous film-tax incentive program.[6][10] Production lasted 100 consecutive days,[11] and each episode was shot on 35mm film.[12] The crew filmed exterior shots of various constructed sets, including a remote sugarcane field outside of Erath,[13] in addition to real life locations such as Fort Macomb, a nineteenth-century fort located outside of New Orleans.[14]

California was selected as the setting for True Detective's sophomore season. Producers were urged to avoid filming in Los Angeles and, instead, focus on the more obscure regions of the state to "capture a certain psycho-sphere ambiance".[15] Production began in November 2014.[16]

Opening sequence
Led by creative director Patrick Clair, True Detective's title sequences were developed by a collaborative team consisting of three motion-design studios: Santa Monica-based Elastic, Antibody and Breeder, both based in Australia.[17][18][19] For the first season, Clair and his team took a variety of photos of the Louisiana scenery, which became the sequence's backbone.[18] They superimposed these images onto low poly meshes, thanks to the use of various animation and special effects techniques. This was a meticulous process for production, since they wanted to avoid creating a digitized look for the sequence.[19] Once its final cut took form, the team polished it by employing optical glitching and motion distortion technique.[18] True Detective's season one opening theme is "Far from Any Road", an alternative country song originally composed by The Handsome Family for their 2003 album Singing Bones.[19] The Sydney Morning Herald included season one's opening sequence in their list of the "Ten of the Best" title sequences on television.[20]

Clair took a similar approach to creating the title sequence for True Detective's second season. Production used material from a number of photographers, including aerial shots captured by David Maisel.[17] However, unlike season one, season two's title sequence incorporates deep, vivid gold and red color, thereby presenting "that more complicated view of California".[17] Leonard Cohen's "Nevermind" is the season two opening theme, which is a song from Cohen's 2014 album Popular Problems.[21] The theme song's lyrics change with every episode, incorporating different verses from Cohen's song.[22][23]

Cast and crew
Season 1
Main article: True Detective (season 1)

Harrelson (left) and McConaughey (right) at the 66th Primetime Emmy Awards.
The first actor to be cast for True Detective was Matthew McConaughey, who acted as Detective Rustin "Rust" Cohle. McConaughey came to Pizzolatto's attention for his performance in the 2011 thriller film The Lincoln Lawyer, and was contracted before the series was commissioned by HBO. He and Woody Harrelson were among a pool of candidates Pizzolatto had in mind for star billing.[5] Although the actor was to play Detective Martin "Marty" Hart, he later convinced Pizzolatto to cast him as Cohle.[24] Instead, Harrelson was assigned the role of Hart at McConaughey's request.[25][26][27] Michelle Monaghan played the female lead Maggie Hart,[28] while Michael Potts and Tory Kittles were given the roles of Detectives Maynard Gilbough and Thomas Papania, respectively.[29][30] Major recurring roles in the first season include Kevin Dunn as Major Ken Quesada and Alexandra Daddario as Lisa Tragnetti.[29][31]

Cary Joji Fukunaga was appointed as director of True Detective's first season. He competed with Alejandro González Iñárritu for the position, but Iñárritu dropped out because of other film commitments.[32][33] To prepare, Fukunaga conducted research with a real-life homicide detective of the Louisiana State Police's Criminal Investigations Division.[34] The director brought on Adam Arkapaw as the project cinematographer, and hired Alex DiGerlando, who he worked with on Benh Zeitlin's Glory at Sea (2008), as the production designer.[11]

Season 2
Main article: True Detective (season 2)
In January 2014, Pizzolatto signed a two-year contract extension with HBO, effectively renewing the series for two additional seasons.[35] Much like its predecessor, season 2 of True Detective consists of eight episodes, all written by Pizzolatto.[15] However, the responsibility of directing was assigned to several people. Justin Lin directed the first two episodes.[36] Fukunaga, who directed all of season one, did not return as director; he remains, however, an executive producer,[37] as do McConaughey and Harrelson. Pizzolatto hired fellow novelist Scott Lasser to help develop and write stories for the second half of the season.[37]

The season's first significant casting was Colin Farrell as Detective Raymond "Ray" Velcoro.[38] Vince Vaughn, playing the role of criminal and entrepreneur Frank Semyon, was officially cast toward the end of the month.[36] By November, True Detective's principal cast expanded to include Rachel McAdams as Detective Antigone "Ani" Bezzerides, Taylor Kitsch as California Highway Patrol Officer Paul Woodrugh, and Kelly Reilly as Jordan Semyon, Frank's wife.[39][40]

Season 3
Main article: True Detective (season 3)

Mahershala Ali plays the lead role in the third season.
In August 2017, HBO officially greenlit a third season, which takes place in the Ozarks over three separate time periods. The first two episodes were directed by Jeremy Saulnier; he had been expected to direct the third episode, but had to exit the series due to scheduling conflicts. In March 2018, it was announced that Daniel Sackheim was added as director and that he and Pizzolatto would direct the remaining six episodes.[41] Pizzolatto is the sole writer, with the exceptions of episodes 4 and 6 which he co-wrote with David Milch and Graham Gordy, respectively. Mahershala Ali was cast as the lead character, playing Wayne Hays, a state police detective from northwest Arkansas.[42] Ray Fisher plays Wayne's son, Henry Hays;[43] Carmen Ejogo plays Amelia Reardon, an Arkansas schoolteacher with a connection to two missing children in 1980;[44] Stephen Dorff plays Roland West, an Arkansas State Investigator;[45] Scoot McNairy plays Tom Purcell, the father of the missing children;[46] and Mamie Gummer plays Lucy Purcell, the mother of the missing children.[47]

Future
As of January 2019, Pizzolatto was developing a storyline for a potential fourth season;[48] however, in April, he abandoned that storyline—saving it for another series or film—and started working on a new storyline that he had discussed with an undisclosed actor.[49]

Pizzolatto signed a new first-look deal with FX Productions in January 2020, with HBO reportedly considering proceeding with future installments of True Detective with other writers.[50]

Episodes
Main article: List of True Detective episodes
Season	Episodes	Originally aired	Average viewers
(in millions)
First aired	Last aired
1	8	January 12, 2014	March 9, 2014	2.33[51]
2	8	June 21, 2015	August 9, 2015	2.61[52]
3	8	January 13, 2019	February 24, 2019	1.25[53]
Reception
Critical response
Season 1
True Detective's first season received positive reviews from television critics, with several naming it among the best television dramas of the year.[54] On the review aggregation website Rotten Tomatoes, the first season garnered a rating of 87%, based on 65 reviews, with an average rating of 8.8/10. The site's critical consensus reads, "In True Detective, performances by Woody Harrelson and Matthew McConaughey reel the viewer in, while the style, vision and direction make it hard to turn away."[55] On Metacritic, season one scored an 87 out of 100, based on 41 critics, indicating "universal acclaim".[56]

Reviewers from The Daily Beast, The Atlantic, and The Daily Telegraph cited True Detective as the strongest show in recent memory.[57][58][59] Tim Goodman from The Hollywood Reporter said the acting, dialogue, and sleek production were the series' most redeeming qualities.[60] HitFix's Alan Sepinwall agreed, and believed that these attributes "speak to the value of the hybrid anthology format Pizzolatto is using here ... points to a potentially fascinating shift in dramatic series television."[61] Richard Lawson, writing for Vanity Fair, said that Pizzolatto and Fukunaga's sensibilities produce "a captivating and offbeat tweak of a well-worn genre".[62] Despite its critical regard, some critics were not as enthusiastic in their reviews of season one. The New York Times journalist Mike Hale thought the script too readily deferred to religion as its narrative backbone,[63] as did Chris Cabin from Slant Magazine.[64] Hank Steuver of The Washington Post wrote that True Detective failed to realize its own ambition.[65]

The ensemble performances, namely McConaughey and Harrelson, were frequently mentioned in review for the show. Robert Bianco in USA Today wrote that the duo met, and even exceeded occasionally, the "enormously high" performance expectations of the "golden age of TV acting".[66] David Wiegand of the San Francisco Chronicle and Los Angeles Times journalist Robert Lloyd singled out the two men for their work in the series;[67][68] The Boston Globe did the same for Monaghan.[69] Variety's Brian Lowry said the True Detective cast consisted of "fine players on the periphery".[70]

Season 2
True Detective's second season received mixed to negative reviews. Praise was given to the performances of Farrell, McAdams and Kitsch,[71] cinematography,[72] and action sequences.[73] However, many critics felt it was weaker than the first season. Most criticism focused on the convoluted plot and dialogue.[74][75] On Rotten Tomatoes, the season has a rating of 64%, based on 72 reviews, with an average rating of 6.6/10. The site's critical consensus reads, "True Detective's second season stands on its own as a solid police drama, with memorable moments and resonant relationships outweighing predictable plot twists."[76] On Metacritic, the season has a score of 61 out of 100, based on 41 critics, indicating "generally favorable reviews".[77]

David Hinckley of the New York Daily News gave it a very positive review, and wrote: "It's still the kind of show that makes TV viewers reach for phrases like 'golden age of television drama'" and that "the second installment of True Detective goes out of the way not to echo the first."[78] Hank Stuever of The Washington Post gave it a generally positive review, praising the performances, and wrote: "There is something still lugubrious and overwrought about True Detective, but there's also a mesmerizing style to it — it's imperfect, but well made."[79]

A more mixed review came from Brian Lowry of Variety, who wrote: "Although generally watchable, the inspiration that turned the first [season] into an obsession for many seems to have drained out of writer Nic Pizzolatto's prose."[80]

Sean T. Collins of Rolling Stone gave it a negative review and described the season as having "emerged as the year's most passionately disliked show," and described it as a "squandered opportunity" for Nic Pizzolatto.[81]

Season 3
The third season received much stronger reviews in comparison to the previous and was marked as a return to form for the series.

On Rotten Tomatoes, the season has a rating of 85%, based on 80 reviews, with an average rating of 7.41/10. The site's critical consensus reads, "Driven by Mahershala Ali's mesmerizing performance, True Detective's third season finds fresh perspective by exploring real world events – though it loses some of the series' intriguing strangeness along the way."[82] On Metacritic, the season has a score of 71 out of 100, based on 32 critics, indicating "generally favorable reviews".[83]

Ratings
True Detective : U.S. viewers per episode (millions)
Audience measurement performed by Nielsen Media Research.[84]
With an average gross audience of 11.9 million viewers, the first season of True Detective was HBO's most-watched show in its first year of airing, surpassing Six Feet Under’s 11.4 million viewers in 2001.[85] This rating system counts all airings of a show in its average audience rating, no matter how many times a viewer watches the show during the report period (including DVR recordings, reruns, and streaming on HBO Go).[86][87] The season averaged 2.33 total million live plus same day viewers, with the viewership numbers keeping a generally upward trend over the course of the season.[88] The premiere was the second most-watched series debut for the network with 2.3 million viewers, behind Boardwalk Empire's 4.8 million in 2010.[89] The sixth episode of the season was watched by 2.6 million viewers, and became the first episode to surpass the premiere, while the season-finale was watched by 3.5 million viewers, which is a 50% increase over the premiere's ratings.[85]

The second-season premiere sustained the viewership numbers of the first season, as the episode was watched by 3.2 million viewers.[89] However, the season's ratings kept a generally downward trend, as the season-finale was watched by 2.7 million viewers, a 22% drop in comparison to the final episode of the first season. Nevertheless, it was the most-watched telecast for the day it aired, drawing an additional 550,000 total viewers from the season's penultimate episode.[90] Overall, the second season of the series averaged 2.61 million total viewers,[91] and 11.3 million viewers in gross audience numbers which is only slightly below the first season's 11.9 million.[92]

Accolades

Season 1 director Cary Joji Fukunaga won the Primetime Emmy Award for Outstanding Directing for a Drama Series.
Main article: List of awards and nominations received by True Detective
For the 30th TCA Awards, True Detective won for Outstanding Achievement in Movies, Miniseries, and Specials and received nominations for Outstanding New Program and Program of the Year; and McConaughey won for Outstanding Individual Achievement in Drama.[93] For the 4th Critics' Choice Television Awards, the series was nominated for Best Drama Series, and McConaughey won for Best Actor in a Drama Series.[94] For the 66th Primetime Emmy Awards, the series was nominated for Outstanding Drama Series, Harrelson and McConaughey were both nominated for Outstanding Lead Actor in a Drama Series, Pizzolatto was nominated for Outstanding Writing for a Drama Series for "The Secret Fate of All Life", and Fukunaga won for Outstanding Directing for a Drama Series for "Who Goes There".[95]

For the 66th Primetime Creative Arts Emmy Awards, the series was nominated for seven awards, and won four, including Outstanding Cinematography for a Single-Camera Series and Outstanding Main Title Design.[96] For the 67th Writers Guild of America Awards, the series won for Best Drama Series and Best New Series.[97] For the 21st Screen Actors Guild Awards, Harrelson and McConaughey both received nominations for Best Drama Actor.[98] For the 72nd Golden Globe Awards, the series was nominated for Best Miniseries or Television Film, Harrelson and McConaughey were both nominated for Best Actor – Miniseries or Television Film, and Monaghan was nominated for Best Supporting Actress – Series, Miniseries or Television Film.[99] For the 67th Directors Guild of America Awards, Fukunaga was nominated for Outstanding Directing – Drama Series for the episode "Who Goes There".[100]

Home media
Season	DVD release date	Blu-ray release date	Ref.
Region 1	Region 2	Region 4	Region A	Region B
1	June 10, 2014	June 9, 2014	June 25, 2014	June 10, 2014	June 9, 2014	[101]
2	January 5, 2016	January 11, 2016	January 6, 2016	January 5, 2016	January 11, 2016	[102]
3	September 3, 2019	September 2, 2019	September 4, 2019	September 3, 2019	September 2, 2019	[103]
References
 Ausiello, Michael (April 9, 2015). "True Detective Season 2 Trailer Is Here!". TVLine. Archived from the original on April 10, 2015. Retrieved April 9, 2015.
 Kim, Monica (June 17, 2015). "Meet the Troubled Cast of True Detective's Season 2". Vogue. Archived from the original on June 18, 2015. Retrieved June 18, 2015.
 Walker, Dave (July 7, 2013). "Nic Pizzolatto, New Orleans-born novelist, discusses HBO's upcoming 'True Detective'". The Times-Picayune. Ricky Matthews. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Romano, Andrew (April 4, 2014). "Inside the Obsessive, Strange Mind of True Detective's Nic Pizzolatto". The Daily Beast. The Newsweek Daily Beast Company. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Goldberg, Lesley (January 9, 2014). "'True Detective': Five Things to Know About HBO's New Cop Drama". The Hollywood Reporter. Prometheus Global Media. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Duelund, Theis (January 4, 2014). "Q&A: "True Detective" Creator Nic Pizzolatto is a One-Man Writing Army". Los Angeles. Emmis Communications. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Navae, Helene (August 23, 2014). "Til masterclass med Mr. True Detective" [To master class with Mr. True Detective]. Politiken (in Danish). JP/Politikens Hus. Archived from the original on November 4, 2015. Retrieved January 26, 2015.
 Andreeva, Nellie (April 30, 2012). "HBO Picks Up Matthew-Woody Series 'True Detective' With Eight-Episode Orders". Deadline Hollywood. Archived from the original on November 4, 2015. Retrieved February 2, 2013.
 Goldberg, Lesley (April 30, 2013). "HBO Lands Matthew McConaughey, Woody Harrelson Cop Drama". The Hollywood Reporter. Archived from the original on November 5, 2013. Retrieved September 9, 2013.
 White, Jr., Lamar (January 16, 2014). "Exclusive: Nic Pizzolatto's Louisiana". CenLamar. Archived from the original on November 5, 2015. Retrieved December 25, 2014.
 Hart, Hugh (January 6, 2014). "Director Cary Fukunaga on Conjuring the Louisiana Noir of "True Detective"". Fast Company. Fast Company, Inc. Archived from the original on November 5, 2015. Retrieved December 25, 2014.
 Cary Joji Fukunaga (director) (2014). "Making True Detective" featurette (Blu-ray). HBO. Event occurs at 7:51–7:53.
 Martin, Denise (March 6, 2014). "Devil's Nests and Beer-Can Men: The Origins of 13 True Detective Set Pieces". New York. New York Media. Archived from the original on November 5, 2015. Retrieved December 25, 2014.
 Martin, Denise (March 10, 2014). "True Detective's Production Designer on the Fort". New York. New York Media. Archived from the original on November 5, 2015. Retrieved January 24, 2015.
 Hibberd, James (May 27, 2014). "'True Detective' season 2 scoop: Trio of leads, California setting". Entertainment Weekly. Archived from the original on May 28, 2014. Retrieved May 27, 2014.
 "Rachel McAdams, Taylor Kitsch and Kelly Reilly Join Colin Farrell and Vince Vaughn in HBO's "True Detective"" (Press release). HBO. November 24, 2014. Retrieved December 2, 2014.
 Horn, John (June 29, 2015). "True Detective's Title Designer on Bringing the Soul of the Show to Its Opening Moments". New York. New York Media. Archived from the original on August 2, 2015. Retrieved August 8, 2015.
 Perkins, Will (January 14, 2014). "True Detective (2014)". Art of the Title. Archived from the original on November 5, 2015. Retrieved January 24, 2015.
 Rhodes, Margaret (March 7, 2014). "A Behind-The-Scenes Look At The Chilling Opening Credits Of "True Detective"". Fast Company. Fast Company, Inc. Archived from the original on November 5, 2015. Retrieved January 24, 2015.
 Kalina, Paul (August 22, 2014). "How Patrick Clair won the Emmy for 90 seconds of True Detective". The Sydney Morning Herald. Fairfax Media. Archived from the original on November 5, 2015. Retrieved January 24, 2015.
 Zoladz, Lindsay (June 21, 2015). "A History of True Detective's New Theme Song, 'Nevermind'". Vulture. Archived from the original on June 23, 2015. Retrieved June 23, 2015.
 Tach, Dave (July 7, 2015). "True Detective season two watchthrough: episodes two and three". Polygon. Archived from the original on July 21, 2015. Retrieved July 28, 2015.
 Pehanick, Maggie (July 6, 2015). "Have You Noticed That True Detective's Theme Song Changes Every Week?". PopSugar. Archived from the original on July 28, 2015. Retrieved July 28, 2015.
 Drumming, Neil (January 9, 2014). ""True Detective" director Cary Fukunaga explains the show's dark humor". Salon. Salon Media Group. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Ringen, Jonathan (March 3, 2014). "Matthew McConaughey on 'True Detective,' His Pal Woody, McConaissance". Rolling Stone. Jann Wenner. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 MacInnes, Paul (March 21, 2014). "True Detective: how we made the most talked-about TV show of the year". The Guardian. Guardian News and Media. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Radish, Christina (January 4, 2014). "Matthew McConaughey and Woody Harrelson Talk True Detective, Why They Agreed to Do a TV Show, Working with Each Other, and More". Collider. Archived from the original on November 4, 2015. Retrieved January 22, 2015.
 Radish, Christina (February 12, 2014). "Michelle Monaghan Talks True Detective, Joining the Show, Working with Cary Fukunaga, the Mood on Set, Future Projects, and More". Collider. Archived from the original on November 4, 2015. Retrieved January 22, 2015.'''

In [271]:
test_algorithm(text_3, 'aalmaraan')

Found aalmaraan on iteration 0
Found aalmaraan on iteration 2
Found aalmaraan on iteration 4
Found aalmaraan on iteration 6
Found aalmaraan on iteration 8
Found aalmaraan on iteration 10
Found aalmaraan on iteration 12
Found aalmaraan on iteration 14
Found aalmaraan on iteration 16
Found aalmaraan on iteration 18
Found aalmaraan on iteration 20
Found aalmaraan on iteration 22
Found aalmaraan on iteration 24
Found aalmaraan on iteration 26
Found aalmaraan on iteration 28
Found aalmaraan on iteration 30
Found aalmaraan on iteration 32
Found aalmaraan on iteration 34
Found aalmaraan on iteration 36
Found aalmaraan on iteration 38
Found aalmaraan on iteration 40
Found aalmaraan on iteration 42
Found aalmaraan on iteration 44
Found aalmaraan on iteration 46
Found aalmaraan on iteration 48
Found aalmaraan on iteration 50
Found aalmaraan on iteration 52
Found aalmaraan on iteration 54
Found aalmaraan on iteration 56
Found aalmaraan on iteration 58
Found aalmaraan on iteration 60
Found aalmara

In [272]:
test_algorithm(text_3, 'saintpetersburg')

Found saintpetersburg on iteration 0
Found saintpetersburg on iteration 3
Found saintpetersburg on iteration 6
Found saintpetersburg on iteration 9
Found saintpetersburg on iteration 12
Found saintpetersburg on iteration 15
Found saintpetersburg on iteration 18
Found saintpetersburg on iteration 21
Found saintpetersburg on iteration 24
Found saintpetersburg on iteration 27
Found saintpetersburg on iteration 30
Found saintpetersburg on iteration 33
Found saintpetersburg on iteration 36
Found saintpetersburg on iteration 39
Found saintpetersburg on iteration 42
Found saintpetersburg on iteration 45
Found saintpetersburg on iteration 48
Found saintpetersburg on iteration 51
Found saintpetersburg on iteration 54
Found saintpetersburg on iteration 57
Found saintpetersburg on iteration 60
Found saintpetersburg on iteration 63
Found saintpetersburg on iteration 66
Found saintpetersburg on iteration 69
Found saintpetersburg on iteration 72
Found saintpetersburg on iteration 75
Found saintpeter